In [1]:
from mido import Message, MidiFile, MidiTrack
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import random

#read signals
source = [
    MidiFile('./data/symphony_9_1.mid'), 
    MidiFile('./data/symphony_9_2.mid'),
    MidiFile('./data/symphony_9_3.mid'),
    MidiFile('./data/symphony_9_4.mid')
]
fileCount = 0
for file in source:
    metadata = []
    originalnotelist = []
    randomnotespool = []
    trackCount = 0
    
    for i, track in enumerate(file.tracks):
        if trackCount % 25 == 0:
            print("track " + str(trackCount) + ' through ' + str(trackCount + 25) + ' being read')
        trackCount += 1
        for m in track:
            if (m.type == 'note_on' or m.type == 'note_off'):
                originalnotelist.append({'track': trackCount, 'type': m.type, 'channel': m.channel, 'note': m.note, 'time': m.time, 'velocity': 127 })#m.velocity })
                randomnotespool.append({'track': trackCount, 'type': m.type, 'channel': m.channel, 'note': random.random() * 127, 'time': random.random() * 100 + 50, 'velocity': 127 })
    
    #learn signals
    print('prepping data')
    notes = pd.DataFrame.from_records(originalnotelist)
    notesfortraining = notes.filter(['note', 'channel', 'time', 'velocity'])
    big_X, little_y = train_test_split(notesfortraining, test_size=0.5)

    # Create and fit a nearest-neighbor classifier
    regression = KNeighborsRegressor(n_neighbors=16)
    print('regression fitting')
    regression.fit(big_X, little_y)

    #predict
    predictednotes = []
    candidates = regression.predict(big_X)
    predictednotes.append(candidates)
    print(str(len(predictednotes[0])) + ' notes predicted...' )

    midifile = MidiFile()
    randomMIDIfile = MidiFile()

    #write predictions to file        
    for t in range(0, trackCount):
        if t % 25 == 0:
                print("track " + str(t) + ' through ' + str(t + 25) + ' being written')
        trackTemp = midifile.add_track("track" + str(t) )
        trackRandom = randomMIDIfile.add_track("track" + str(t) )

        for note in predictednotes[0]:
            note = note.round(0)
            note = note.astype(int)

            trackTemp.append( Message( 'note_on', note=note[0],  channel=min(note[1], 16),  time=max(note[2], 250), velocity=max(note[3], 127) ))

    print('writing to file ' + str(fileCount))
    fileCount += 1 
    midifile.save('./data/Beethoven Approximation - Movement ' + str(fileCount) + '.midi')

print('\nyour music is ready!')

track 0 through 25 being read
prepping data
regression fitting
30935 notes predicted...
track 0 through 25 being written
writing to file 0
track 0 through 25 being read
prepping data
regression fitting
33694 notes predicted...
track 0 through 25 being written
writing to file 1
track 0 through 25 being read
prepping data
regression fitting
8499 notes predicted...
track 0 through 25 being written
writing to file 2
track 0 through 25 being read
track 25 through 50 being read
prepping data
regression fitting
47759 notes predicted...
track 0 through 25 being written
track 25 through 50 being written
writing to file 3
your music is ready!
